In [8]:
# Installing requirements
!pip install --upgrade git+https://github.com/google/flax.git


  Cloning https://github.com/google/flax.git to c:\users\yashk\appdata\local\temp\pip-req-build-9tji1fwc
  Resolved https://github.com/google/flax.git to commit b8ccb153f29c42077021e770bdb3f7c89de6affc
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached jax-0.4.26-py3-none-any.whl.metadata (23 kB)
  Using cached msgpack-1.0.8-cp312-cp312-win_amd64.whl.metadata (9.4 kB)
  Using cached optax-0.2.2-py3-none-any.whl.metadata (8.1 kB)
  Using cached orbax_checkpoint-0.5.10-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorstore-0.1.56-cp312-cp312-win_amd64.whl.metadata (3.0 kB)
  Using cached 

  Running command git clone --filter=blob:none --quiet https://github.com/google/flax.git 'C:\Users\yashk\AppData\Local\Temp\pip-req-build-9tji1fwc'


In [4]:
!pip install Pyqt5

   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/6.8 MB 656.4 kB/s eta 0:00:11
   - -------------------------------------- 0.3/6.8 MB 2.5 MB/s eta 0:00:03
   - -------------------------------------- 0.3/6.8 MB 2.5 MB/s eta 0:00:03
   -- ------------------------------------- 0.5/6.8 MB 2.4 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/6.8 MB 2.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/6.8 MB 2.5 MB/s eta 0:00:03
   ---- ----------------------------------- 0.7/6.8 MB 2.4 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/6.8 MB 2.6 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/6.8 MB 2.6 MB/s eta 0:00:03
   ----- ---------------------------------- 0.9/6.8 MB 2.1 MB/s eta 0:00:03
   ----- ----------------------

In [1]:
from transformers import FlaxAutoModelForSeq2SeqLM, AutoTokenizer

# Path to the local tokenizer directory
LOCAL_TOKENIZER_PATH = r"C:\Users\yashk\OneDrive\Desktop\Project\t5-recipe-generation"

# Path to the local model directory
LOCAL_MODEL_PATH = r"C:\Users\yashk\OneDrive\Desktop\Project\t5-recipe-generation"

# Load tokenizer from the local directory
tokenizer = AutoTokenizer.from_pretrained(LOCAL_TOKENIZER_PATH, use_fast=True)

# Load model from the local directory
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(LOCAL_MODEL_PATH)

# Rest of your code remains the same
prefix = "items: "
generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}
# Define skip_special_tokens and target_postprocessing functions as before

def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]
    
    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs, 
        max_length=256, 
        padding="max_length", 
        truncation=True, 
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids, 
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe

# Example usage
items = [
    "macaroni, butter, salt, bacon, milk, flour, pepper, cream corn",
    "provolone cheese, bacon, bread, ginger"
]
generated = generation_function(items)
for text in generated:
    print(text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 title: mom s mac n cheese
 ingredients: 2 cups macaroni-- 1 tablespoon butter-- 1 teaspoon salt-- 6 pieces bacon-- 4 cups milk-- 1 cup flour-- 14 teaspoon pepper-- 1 9 ounce can cream corn-- 1 11 ounce jar spaghetti sauce your favorite
 directions: cook macaronine according to package directions and drain.-- brown bacon until slightly crispy.-- add flour, salt, and pepper to drippings and stir until smooth.-- cook mixture for 3 minutes or until it becomes bubbly.-- gradually add milk.-- heat mixture to boiling and cook for a few minutes until thickened.-- remove from heat.-- return macaronini to pot, stir in corn and bacon.-- pour into a greased casserole and top with spaghetti sauce.-- bake at 350 degrees for about 30 minutes or longer if desired.
 title: hot dog pizza
 ingredients: 1 lb hot dog-- provolone cheese-- 12 slices bacon, cooked and crumbled-- 6 slices multigrain bread-- pickled ginger
 directions: cut each hot dog in half lengthwise.-- stuff with a slice of cheese and a p

In [3]:
"""from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer

MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

prefix = "items: "
# generation_kwargs = {
#     "max_length": 512,
#     "min_length": 64,
#     "no_repeat_ngram_size": 3,
#     "early_stopping": True,
#     "num_beams": 5,
#     "length_penalty": 1.5,
# }
generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}


special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]
    
    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs, 
        max_length=256, 
        padding="max_length", 
        truncation=True, 
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids, 
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe
"""

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


flax_model.msgpack:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [2]:
items = [
    "macaroni, butter, salt, bacon, milk, flour, pepper, cream corn",
    "provolone cheese, bacon, bread, ginger"
]
generated = generation_function(items)
for text in generated:
    sections = text.split("\n")
    for section in sections:
        section = section.strip()
        if section.startswith("title:"):
            section = section.replace("title:", "")
            headline = "TITLE"
        elif section.startswith("ingredients:"):
            section = section.replace("ingredients:", "")
            headline = "INGREDIENTS"
        elif section.startswith("directions:"):
            section = section.replace("directions:", "")
            headline = "DIRECTIONS"
        
        if headline == "TITLE":
            print(f"[{headline}]: {section.strip().capitalize()}")
        else:
            section_info = [f"  - {i+1}: {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
            print(f"[{headline}]:")
            print("\n".join(section_info))

    print("-" * 130)


[TITLE]: Mom s mac n cheese
[INGREDIENTS]:
  - 1: 2 cups macaroni
  - 2: 1 tablespoon butter
  - 3: 1 teaspoon salt
  - 4: 6 pieces bacon
  - 5: 4 cups milk
  - 6: 1 cup flour
  - 7: 14 teaspoon pepper
  - 8: 1 9 ounce can cream corn
  - 9: 1 11 ounce jar spaghetti sauce your favorite
[DIRECTIONS]:
  - 1: Cook macaronine according to package directions and drain.
  - 2: Brown bacon until slightly crispy.
  - 3: Add flour, salt, and pepper to drippings and stir until smooth.
  - 4: Cook mixture for 3 minutes or until it becomes bubbly.
  - 5: Gradually add milk.
  - 6: Heat mixture to boiling and cook for a few minutes until thickened.
  - 7: Remove from heat.
  - 8: Return macaronini to pot, stir in corn and bacon.
  - 9: Pour into a greased casserole and top with spaghetti sauce.
  - 10: Bake at 350 degrees for about 30 minutes or longer if desired.
----------------------------------------------------------------------------------------------------------------------------------
[TITLE

In [5]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QTextEdit, QPushButton, QVBoxLayout

class RecipeGenerator(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Recipe Generator')
        self.setGeometry(100, 100, 600, 400)

        self.input_label = QLabel('Enter ingredients (separated by commas):')
        self.input_textedit = QTextEdit()
        self.generate_button = QPushButton('Generate Recipe')
        self.generate_button.clicked.connect(self.generate_recipe)
        self.output_label = QLabel('Generated Recipe:')
        self.output_textedit = QTextEdit()
        self.output_textedit.setReadOnly(True)

        layout = QVBoxLayout()
        layout.addWidget(self.input_label)
        layout.addWidget(self.input_textedit)
        layout.addWidget(self.generate_button)
        layout.addWidget(self.output_label)
        layout.addWidget(self.output_textedit)

        self.setLayout(layout)

    def generate_recipe(self):
        items = self.input_textedit.toPlainText().split(',')
        generated = generation_function(items)
        recipe_text = ''
        for text in generated:
            sections = text.split("\n")
            for section in sections:
                section = section.strip()
                if section.startswith("title:"):
                    section = section.replace("title:", "")
                    headline = "TITLE"
                elif section.startswith("ingredients:"):
                    section = section.replace("ingredients:", "")
                    headline = "INGREDIENTS"
                elif section.startswith("directions:"):
                    section = section.replace("directions:", "")
                    headline = "DIRECTIONS"

                if headline == "TITLE":
                    recipe_text += f"[{headline}]: {section.strip().capitalize()}\n\n"
                else:
                    section_info = [f"  - {i+1}: {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
                    recipe_text += f"[{headline}]:\n"
                    recipe_text += "\n".join(section_info) + "\n\n"

                recipe_text += "-" * 130 + "\n"

        self.output_textedit.setPlainText(recipe_text)


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = RecipeGenerator()
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\yashk\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QTextEdit, QPushButton, QVBoxLayout

class RecipeGenerator(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Recipe Generator')
        self.setGeometry(100, 100, 800, 600)

        self.input_label = QLabel('Enter ingredients (separated by commas):')
        self.input_textedit = QTextEdit()
        self.generate_button = QPushButton('Generate Recipe')
        self.generate_button.clicked.connect(self.generate_recipe)
        self.output_label = QLabel('Generated Recipe:')
        self.output_textedit = QTextEdit()
        self.output_textedit.setReadOnly(True)

        layout = QVBoxLayout()
        layout.addWidget(self.input_label)
        layout.addWidget(self.input_textedit)
        layout.addWidget(self.generate_button)
        layout.addWidget(self.output_label)
        layout.addWidget(self.output_textedit)

        self.setLayout(layout)

    def generate_recipe(self):
        items = self.input_textedit.toPlainText().split(',')
        generated = generation_function(items)
        recipe_text = ''
        for text in generated:
            sections = text.split("\n")
            for section in sections:
                section = section.strip()
                if section.startswith("title:"):
                    section = section.replace("title:", "")
                    recipe_text += f"<b>{section.strip().capitalize()}</b>\n\n"
                elif section.startswith("ingredients:"):
                    section = section.replace("ingredients:", "")
                    recipe_text += f"<u>Ingredients:</u>\n"
                    ingredients = section.split("--")
                    for i, ingredient in enumerate(ingredients, 1):
                        recipe_text += f"  - {i}: {ingredient.strip().capitalize()}\n"
                    recipe_text += "\n"
                elif section.startswith("directions:"):
                    section = section.replace("directions:", "")
                    recipe_text += f"<u>Directions:</u>\n"
                    directions = section.split("--")
                    for i, direction in enumerate(directions, 1):
                        recipe_text += f"  - {i}: {direction.strip().capitalize()}\n"
                    recipe_text += "\n"

            recipe_text += "-" * 130 + "\n"

        self.output_textedit.setHtml(recipe_text)


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = RecipeGenerator()
    window.show()
    sys.exit(app.exec_())


NameError: name 'generation_function' is not defined

SystemExit: 0

C:\Users\yashk\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
import tkinter as tk
from tkinter import ttk

def generate_recipe(text_widget, output_widget):
    items = text_widget.get("1.0", "end-1c").split("\n")
    try:
        generated = generation_function(items)
        output_widget.config(state=tk.NORMAL)
        output_widget.delete("1.0", tk.END)
        for text in generated:
            sections = text.split("\n")
            for section in sections:
                section = section.strip()
                if section.startswith("title:"):
                    section = section.replace("title:", "")
                    headline = "TITLE"
                elif section.startswith("ingredients:"):
                    section = section.replace("ingredients:", "")
                    headline = "INGREDIENTS"
                elif section.startswith("directions:"):
                    section = section.replace("directions:", "")
                    headline = "DIRECTIONS"

                if headline == "TITLE":
                    output_widget.insert(tk.END, f"[{headline}]: {section.strip().capitalize()}\n\n")
                else:
                    section_info = [f"  - {i+1}: {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
                    output_widget.insert(tk.END, f"[{headline}]:\n")
                    output_widget.insert(tk.END, "\n".join(section_info) + "\n\n")

                output_widget.insert(tk.END, "-" * 130 + "\n")

        output_widget.config(state=tk.DISABLED)
    except Exception as e:
        print("Error:", e)  # Print any errors that occur

# Initialize Tkinter
root = tk.Tk()
root.title("Recipe Generator")

# Styling
style = ttk.Style()
style.configure("Title.TLabel", font=("Helvetica", 16, "bold"))
style.configure("Text.TLabel", font=("Helvetica", 12))
style.configure("Text.TButton", font=("Helvetica", 12))

# Input area for entering ingredients
label = ttk.Label(root, text="Enter ingredients one by one:", style="Title.TLabel")
label.pack()

text_widget = tk.Text(root, height=10, width=50, font=("Helvetica", 12))
text_widget.pack()

# Output area for displaying generated recipe
output_widget = tk.Text(root, height=30, width=100, state=tk.DISABLED, font=("Helvetica", 12))
output_widget.pack()

# Button to generate recipe
generate_button = ttk.Button(root, text="Generate Recipe", command=lambda: generate_recipe(text_widget, output_widget), style="Text.TButton")
generate_button.pack()

root.mainloop()


In [ ]:
from flask import Flask, render_template, request
import generation_function

app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        items = request.form.get("items").split("\n")
        generated = generation_function(items)
        return render_template("result.html", generated=generated)
    return render_template("index.html")

if __name__ == "__main__":
    app.run(debug=True)


In [3]:
from transformers import DetrForObjectDetection, DetrImageProcessor
import torch
import cv2

# Load the processor and model
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

# Load and preprocess the image
image_path = r"C:\Users\yashk\Downloads\pexels-enginakyurt-1435904.jpg"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
inputs = processor(images=image, return_tensors="pt")

# Run the model
outputs = model(**inputs)

# Get the predicted class logits and bounding boxes
pred_logits = outputs.logits
pred_boxes = outputs.pred_boxes

# Get the class labels from the model's configuration
labels = list(model.config.id2label.values())

# Display the detected objects and their labels
for logits, boxes in zip(pred_logits, pred_boxes):
    class_idx = logits.argmax(-1).cpu().numpy()[0]  # Convert to numpy array and access the first element
    class_label = labels[class_idx]
    print(f"Detected object: {class_label}, Bounding box: {boxes}")


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: a Tensor with 100 elements cannot be converted to Scalar

In [7]:
!pip install timm


In [2]:
import sys
print(sys.executable)


C:\Users\yashk\AppData\Local\Programs\Python\Python312\python.exe
